In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay
from lsst.geom import Point2D

In [ ]:
def displayExposure(exposure,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    afwDisplay.setImageColormap(cmap='gray')
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(exposure.getImage())
    plt.title(title)
    plt.gca().axis('off')
    return afw_display

def displayImage(image,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    #afw_display.scale('linear', min=-5, max=10)
    afw_display.setImageColormap(cmap='plasma')
    afw_display.mtv(image)
    plt.title(title)
    plt.gca().axis('off')
    return afw_display
    
def displayImageGhosts(image, zmin=0, zmax=5000, title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    #afw_display.scale('asinh', 'zscale')
    afw_display.scale('linear', min=zmin, max=zmax)
    afw_display.setImageColormap(cmap='plasma')
    afw_display.mtv(image)
    plt.title(title)
    plt.gca().axis('off')
    return afw_display    

In [ ]:
# Define butler
from lsst.daf.butler import Butler
butler = Butler('/repo/main', collections=['LSSTComCam'])


In [ ]:
def getCentralWcs(expo="2024102400128"):
    calexp_refs = butler.query_datasets("preliminary_visit_image", where=f"instrument='LSSTComCam' AND exposure={expo}")
    return butler.get(calexp_refs[4]).getWcs()

def getStarWcs(expo="2024102400128"):
    calexp_refs = butler.query_datasets("preliminary_visit_image", where=f"instrument='LSSTComCam' AND exposure={expo}")
    return butler.get(calexp_refs[3]).getWcs()

def getAllWcs(expo="2024102400128"):
    calexp_refs = butler.query_datasets("preliminary_visit_image", where=f"instrument='LSSTComCam' AND exposure={expo}")
    wcs_list = [butler.get(calexp_refs[i]).getWcs() for i in range(9)]
    return wcs_list

def queryExposures(expo="2024102400128"):
    dataset_refs = butler.query_datasets("post_isr_image", where=f"instrument='LSSTComCam' AND exposure={expo}")
    exposures = [butler.get(dataset_ref) for dataset_ref in dataset_refs]
    return exposures

def make_mosaic(exposures, binning=4):
    from lsst.pipe.tasks.visualizeVisit import (
        VisualizeBinExpConfig,
        VisualizeBinExpTask,
        VisualizeMosaicExpConfig,
        VisualizeMosaicExpTask,
    )
    camera = butler.get("camera", collections="LSSTComCam/defaults", instrument="LSSTComCam")
    
    visualizeBinExpConfig = VisualizeBinExpConfig()
    visualizeBinExpConfig.binning = binning
    visualizeBinExpTask = VisualizeBinExpTask(config=visualizeBinExpConfig)
    exposures_binned = [visualizeBinExpTask.run(inputExp = exposure, camera=camera).outputExp for exposure in exposures]
    
    visualizeMosaicExpConfig = VisualizeMosaicExpConfig()
    visualizeMosaicExpConfig.binning = binning
    visualizeMosaicExpTask = VisualizeMosaicExpTask(config=visualizeMosaicExpConfig)
    
    mosaic_full = visualizeMosaicExpTask.run(inputExps=exposures_binned, camera=camera)
    mosaic = mosaic_full.outputData
    return mosaic

def show_ghosts(exp_id="2024102400126", binning=4, zmin=2230, zmax=2330):
    expos = queryExposures(exp_id)
    mosaic = make_mosaic(expos, binning)
    displayImageGhosts(mosaic, zmin=zmin, zmax=zmax)
    return mosaic, expos

In [ ]:
from lsst.daf.butler import Butler
from lsst.pipe.tasks.visualizeVisit import (
    VisualizeBinExpConfig,
    VisualizeBinExpTask,
    VisualizeMosaicExpConfig,
    VisualizeMosaicExpTask,
)

butler = Butler.from_config("/repo/main")


def make_mosaic(exposures, binning=4):
    visualizeBinExpConfig = VisualizeBinExpConfig()
    visualizeBinExpConfig.binning = binning
    visualizeBinExpTask = VisualizeBinExpTask(config=visualizeBinExpConfig)

    camera = butler.get("camera", collections="LSSTComCam/DP1", instrument="LSSTComCam")
    exposures_binned = [visualizeBinExpTask.run(exposure, camera).outputExp for exposure in exposures]

    visualizeMosaicExpConfig = VisualizeMosaicExpConfig()
    visualizeMosaicExpConfig.binning = binning
    visualizeMosaicExpTask = VisualizeMosaicExpTask(config=visualizeMosaicExpConfig)

    mosaic_full = visualizeMosaicExpTask.run(inputExps=exposures_binned, camera=camera)
    mosaic = mosaic_full.outputData
    return mosaic


def show_ghosts(exp_id=2024112600112, binning=4, zmin=645, zmax=689):
    exposure_refs = butler.query_datasets(
        "post_isr_image",
        collections="LSSTComCam/nightlyValidation",
        find_first=False,
        where=f"instrument='LSSTComCam' AND exposure={exp_id}",
    )
    exposures = [butler.get(exposure_ref) for exposure_ref in exposure_refs]
    mosaic = make_mosaic(exposures, binning)
    displayImageGhosts(mosaic, zmin=zmin, zmax=zmax)
    return mosaic, exposures

In [ ]:
expos = "2024102400126"

In [ ]:
mosaic = show_ghosts(exp_id="2024102400126", binning=1, zmin=2230, zmax=2330)

In [ ]:
displayImageGhosts(mosaic, zmin=820, zmax=900)

In [ ]:
mosaic[0].writeFits(f"{expos}.fits", mosaic[1][0].getMetadata(), "w")

In [ ]:
mosaic[0].getDimensions()

# PostISR are delete from Butler